<a href="https://colab.research.google.com/github/dhruszecki/cdatos-st-tp/blob/main/turing-preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import os
import os.path
from os import path

import matplotlib.pyplot as plt

from enum import Enum
from time import time




# Utils

In [ ]:
#importamos el json en crudo desde el site de convai
raw_dataset = pd.read_json("http://convai.io/data/summer_wild_evaluation_dialogs.json", convert_dates=True, lines=True)


In [ ]:
#imprimimos la columna 0 para ver el formato (hay 3495 columnas, donde cada una es una conversacion)
column0=pd.json_normalize(raw_dataset[0])
column0

,dialog_id,dialog,bot_profile,user_profile,eval_score,profile_match
0,0xab38710,"[{'id': 0, 'sender': 'participant2', 'text': 'I love iphone! i just bought new iphone!', 'sender_class': 'Bot'}, {'id': 1, 'sender': 'participant1', 'text': 'Thats good for you, i'm not very into new tech', 'sender_class': 'Human'}, {'id': 2, 'sender': 'participant2', 'text': 'I am a college student and i am a college student', 'sender_class': 'Bot'}, {'id': 3, 'sender': 'participant1', 'text': 'I am go to gym and live on donations', 'sender_class': 'Human'}, {'id': 4, 'sender': 'participant2', 'text': 'I am a vegan and i am in the midwest', 'sender_class': 'Bot'}, {'id': 5, 'sender': 'participant1', 'text': 'So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat', 'sender_class': 'Human'}, {'id': 6, 'sender': 'participant2', 'text': 'I would not mind having them in the office that would be hard for me', 'sender_class': 'Bot'}, {'id': 7, 'sender': 'participant1', 'text': 'Dogs or vegan in office?', 'sender_class': 'Human'}, {'id': 8, 'sender': 'participant2', 'text': 'I am a vegetarian so i am vegan', 'sender_class': 'Bot'}, {'id': 9, 'sender': 'participant1', 'text': 'Strange answer', 'sender_class': 'Human'}, {'id': 10, 'sender': 'participant2', 'text': 'I guess i just do not know what to ps4 experience with.', 'sender_class': 'Bot'}]","[i have amazing children and grandchildren., i can sew my own clothes., i had cancer but its gone now., i am retired and living the great life., i do not have a smartphone.]","[two dogs live with me., i like doing the treadmill and rowing machine., eating is something i do when i'm bored., i have short hair., i go to the gym regularly.]",1,0


In [ ]:
#transponemos
raw_dataset_transposed = raw_dataset.T

In [ ]:
#imprimimos para ver como quedo transpuesto
raw_dataset_transposed.head(5)

,0
0,"{'dialog_id': '0xab38710', 'dialog': [{'id': 0, 'sender': 'participant2', 'text': 'I love iphone! i just bought new iphone!', 'sender_class': 'Bot'}, {'id': 1, 'sender': 'participant1', 'text': 'Thats good for you, i'm not very into new tech', 'sender_class': 'Human'}, {'id': 2, 'sender': 'participant2', 'text': 'I am a college student and i am a college student', 'sender_class': 'Bot'}, {'id': 3, 'sender': 'participant1', 'text': 'I am go to gym and live on donations', 'sender_class': 'Human'}, {'id': 4, 'sender': 'participant2', 'text': 'I am a vegan and i am in the midwest', 'sender_class': 'Bot'}, {'id': 5, 'sender': 'participant1', 'text': 'So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat', 'sender_class': 'Human'}, {'id': 6, 'sender': 'participant2', 'text': 'I would not mind having them in the office that would be hard for me', 'sender_class': 'Bot'}, {'id': 7, 'sender': 'participant1', 'text': 'Dogs or vegan in office?', 'sender_class': 'Human'}, {'id': 8, 'sender': 'participant2', 'text': 'I am a vegetarian so i am vegan', 'sender_class': 'Bot'}, {'id': 9, 'sender': 'participant1', 'text': 'Strange answer', 'sender_class': 'Human'}, {'id': 10, 'sender': 'participant2', 'text': 'I guess i just do not know what to ps4 experience with.', 'sender_class': 'Bot'}], 'bot_profile': ['i have amazing children and grandchildren.', 'i can sew my own clothes.', 'i had cancer but its gone now.', 'i am retired and living the great life.', 'i do not have a smartphone.'], 'user_profile': ['two dogs live with me.', 'i like doing the treadmill and rowing machine.', 'eating is something i do when i'm bored.', 'i have short hair.', 'i go to the gym regularly.'], 'eval_score': 1, 'profile_match': 0}"
1,"{'dialog_id': '0x223f6d5e', 'dialog': [{'id': 0, 'sender': 'participant1', 'text': 'Hello', 'sender_class': 'Human'}, {'id': 1, 'sender': 'participant1', 'text': '/test', 'sender_class': 'Human'}], 'bot_profile': ['my father was a door to door salesman.', 'i've three dogs.', 'i am in an open polyamorous relationship.', 'i also have a dog walking business.'], 'user_profile': ['if i'm not cooking in the kitchen then i'm definitely on the water.', 'spiderman is my favorite.', 'hey my name is larry and i am a chef.', 'l also love comic books.', 'i love cooking but i also enjoy fishing.'], 'eval_score': 1, 'profile_match': 1}"
2,"{'dialog_id': '0x6789b32a', 'dialog': [{'id': 0, 'sender': 'participant1', 'text': 'Hey', 'sender_class': 'Human'}, {'id': 1, 'sender': 'participant1', 'text': 'What do you do?', 'sender_class': 'Human'}], 'bot_profile': ['i am a gold medalist olympian.', 'i love italian food and garlic bread.', 'my favorite color is yellow.', 'gymnastics is my favorite sport.', 'i workout four hours a day.'], 'user_profile': ['i've a german shepherd dog.', 'i live is a rural farming community.', 'my husband is adopted.', 'i like to watch nhl hockey.'], 'eval_score': 1, 'profile_match': 1}"
3,"{'dialog_id': '0x7304a92a', 'dialog': [{'id': 0, 'sender': 'participant2', 'text': 'I love painting, i love to paint', 'sender_class': 'Bot'}], 'bot_profile': ['i fantasize about taking over the world.', 'i'm a 56 year old woman.', 'i have two grown children who never visit.', 'i used to be a painter but now i am a housekeeper.'], 'user_profile': ['i like watching war documentaries.', 'i volunteer at a soup kitchen.', 'cheeseburgers are my favorite food.', 'i was poor growing up.', 'i'm a retired gym teacher.'], 'eval_score': 1, 'profile_match': 1}"
4,"{'dialog_id': '0x7470f02a', 'dialog': [{'id': 0, 'sender': 'participant1', 'text': 'Hello', 'sender_class': 'Human'}], 'bot_profile': ['i am 40 years old.', 'i work as a car salesman.', 'my wife spends all my money.', 'i hate my job.', 'i'm planning on getting a divorce.'], 'user_profile': ['i work for our local supermarket.', 'my favorite band is the who.', 'i have never been out of ohio.', 'my favorite food is pizza with black olives.'], 'eval_sc

In [ ]:
#hacemos la primera conversion desde formato json
formatted_dataset = pd.json_normalize(raw_dataset_transposed[0])

In [ ]:
#nos queda formateado, pero con el dialogo todavia en formato json
pd.set_option('display.max_colwidth', None)
formatted_dataset.head(5)

,dialog_id,dialog,bot_profile,user_profile,eval_score,profile_match
0,0xab38710,"[{'id': 0, 'sender': 'participant2', 'text': 'I love iphone! i just bought new iphone!', 'sender_class': 'Bot'}, {'id': 1, 'sender': 'participant1', 'text': 'Thats good for you, i'm not very into new tech', 'sender_class': 'Human'}, {'id': 2, 'sender': 'participant2', 'text': 'I am a college student and i am a college student', 'sender_class': 'Bot'}, {'id': 3, 'sender': 'participant1', 'text': 'I am go to gym and live on donations', 'sender_class': 'Human'}, {'id': 4, 'sender': 'participant2', 'text': 'I am a vegan and i am in the midwest', 'sender_class': 'Bot'}, {'id': 5, 'sender': 'participant1', 'text': 'So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat', 'sender_class': 'Human'}, {'id': 6, 'sender': 'participant2', 'text': 'I would not mind having them in the office that would be hard for me', 'sender_class': 'Bot'}, {'id': 7, 'sender': 'participant1', 'text': 'Dogs or vegan in office?', 'sender_class': 'Human'}, {'id': 8, 'sender': 'participant2', 'text': 'I am a vegetarian so i am vegan', 'sender_class': 'Bot'}, {'id': 9, 'sender': 'participant1', 'text': 'Strange answer', 'sender_class': 'Human'}, {'id': 10, 'sender': 'participant2', 'text': 'I guess i just do not know what to ps4 experience with.', 'sender_class': 'Bot'}]","[i have amazing children and grandchildren., i can sew my own clothes., i had cancer but its gone now., i am retired and living the great life., i do not have a smartphone.]","[two dogs live with me., i like doing the treadmill and rowing machine., eating is something i do when i'm bored., i have short hair., i go to the gym regularly.]",1.0,0
1,0x223f6d5e,"[{'id': 0, 'sender': 'participant1', 'text': 'Hello', 'sender_class': 'Human'}, {'id': 1, 'sender': 'participant1', 'text': '/test', 'sender_class': 'Human'}]","[my father was a door to door salesman., i've three dogs., i am in an open polyamorous relationship., i also have a dog walking business.]","[if i'm not cooking in the kitchen then i'm definitely on the water., spiderman is my favorite., hey my name is larry and i am a chef., l also love comic books., i love cooking but i also enjoy fishing.]",1.0,1
2,0x6789b32a,"[{'id': 0, 'sender': 'participant1', 'text': 'Hey', 'sender_class': 'Human'}, {'id': 1, 'sender': 'participant1', 'text': 'What do you do?', 'sender_class': 'Human'}]","[i am a gold medalist olympian., i love italian food and garlic bread., my favorite color is yellow., gymnastics is my favorite sport., i workout four hours a day.]","[i've a german shepherd dog., i live is a rural farming community., my husband is adopted., i like to watch nhl hockey.]",1.0,1
3,0x7304a92a,"[{'id': 0, 'sender': 'participant2', 'text': 'I love painting, i love to paint', 'sender_class': 'Bot'}]","[i fantasize about taking over the world., i'm a 56 year old woman., i have two grown children who never visit., i used to be a painter but now i am a housekeeper.]","[i like watching war documentaries., i volunteer at a soup kitchen., cheeseburgers are my favorite food., i was poor growing up., i'm a retired gym teacher.]",1.0,1
4,0x7470f02a,"[{'id': 0, 'sender': 'participant1', 'text': 'Hello', 'sender_class': 'Human'}]","[i am 40 years old., i work as a car salesman., my wife spends all my money., i hate my job., i'm planning on getting a divorce.]","[i work for our local supermarket., my favorite band is the who., i have never been out of ohio., my favorite food is pizza with black olives.]",1.0,1


In [ ]:
#cantidad de dialogos diferentes
index = formatted_dataset.index
number_of_rows = len(index)
print(number_of_rows)

3495


In [ ]:
#imprimimos solo el dialog que esta todavia en formato json
formatted_dataset.dialog.head(10)

0    [{'id': 0, 'sender': 'participant2', 'text': 'I love iphone! i just bought new iphone!', 'sender_class': 'Bot'}, {'id': 1, 'sender': 'participant1', 'text': 'Thats good for you, i'm not very into new tech', 'sender_class': 'Human'}, {'id': 2, 'sender': 'participant2', 'text': 'I am a college student and i am a college student', 'sender_class': 'Bot'}, {'id': 3, 'sender': 'participant1', 'text': 'I am go to gym and live on donations', 'sender_class': 'Human'}, {'id': 4, 'sender': 'participant2', 'text': 'I am a vegan and i am in the midwest', 'sender_class': 'Bot'}, {'id': 5, 'sender': 'participant1', 'text': 'So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat', 'sender_class': 'Human'}, {'id': 6, 'sender': 'participant2', 'text': 'I would not mind having them in the office that would be hard for me', 'sender_class': 'Bot'}, {'id': 7, 'sender': 'participant1', 'text': 'Dogs or vegan in office?', 'sender_class': 'Human'}, {'id': 8, 'sender

In [ ]:
#el primer dialogo con su codigo de id correspondiente, descartando el resto de la información por no ser relevante para nuestro tp
testdialog0=pd.json_normalize(formatted_dataset.dialog[0])
testdialog0['dialog_id']=formatted_dataset.dialog_id[0]
testdialog0.head(1000)


,id,sender,text,sender_class,dialog_id
0,0,participant2,I love iphone! i just bought new iphone!,Bot,0xab38710
1,1,participant1,"Thats good for you, i'm not very into new tech",Human,0xab38710
2,2,participant2,I am a college student and i am a college student,Bot,0xab38710
3,3,participant1,I am go to gym and live on donations,Human,0xab38710
4,4,participant2,I am a vegan and i am in the midwest,Bot,0xab38710
5,5,participant1,So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat,Human,0xab38710
6,6,participant2,I would not mind having them in the office that would be hard for me,Bot,0xab38710
7,7,participant1,Dogs or vegan in office?,Human,0xab38710
8,8,participant2,I am a vegetarian so i am vegan,Bot,0xab38710
9,9,participant1,Strange answer,Human,0xab38710


In [ ]:
#generamos un dataframe con todos los dialogos agregando el dialog_id para agrupar cada conversacion

#creamos el dataframe
df = pd.DataFrame()

#primer dialogo agregado al dataframe convirtiendo cada json de la columna dialog
testdialog_temp=pd.json_normalize(formatted_dataset.dialog[0])
testdialog_temp['dialog_id']=formatted_dataset.dialog_id[0]
df = testdialog_temp

#iteramos con todos los elementos
index = formatted_dataset.index
number_of_rows = len(index)

for i in range(1, number_of_rows):
  testdialog_temp=pd.json_normalize(formatted_dataset.dialog[i])
  testdialog_temp['dialog_id']=formatted_dataset.dialog_id[i]
  df=df.append(testdialog_temp)

In [ ]:
#nos quedan todos los dialogos con id de cada interaccion agrupadas por dialog_id y con la etiqueta correspondiente de 'Bot' o 'Human'
df.head(100)

,id,sender,text,sender_class,dialog_id
0,0,participant2,I love iphone! i just bought new iphone!,Bot,0xab38710
1,1,participant1,"Thats good for you, i'm not very into new tech",Human,0xab38710
2,2,participant2,I am a college student and i am a college student,Bot,0xab38710
3,3,participant1,I am go to gym and live on donations,Human,0xab38710
4,4,participant2,I am a vegan and i am in the midwest,Bot,0xab38710
...,...,...,...,...,...
1,1,participant1,Hi,Human,0x823c044
2,2,participant2,"i like coffee , but i like coffee too",Bot,0x823c044
3,3,participant1,I like drinking tea while reading some books,Human,0x823c044
4,4,participant2,"i like red , reminds me of summer time , which is why i hate it .",Bot,0x823c044


In [ ]:
#cantidad de dialogos diferentes
df['dialog_id'].nunique()

3495

In [ ]:
#cantidad de interacciones
df['dialog_id'].count()

47234

In [ ]:
df

,id,sender,text,sender_class,dialog_id
0,0,participant2,I love iphone! i just bought new iphone!,Bot,0xab38710
1,1,participant1,"Thats good for you, i'm not very into new tech",Human,0xab38710
2,2,participant2,I am a college student and i am a college student,Bot,0xab38710
3,3,participant1,I am go to gym and live on donations,Human,0xab38710
4,4,participant2,I am a vegan and i am in the midwest,Bot,0xab38710
...,...,...,...,...,...
16,16,participant2,"yes , i love to hear voices .",Bot,0x14f93908
17,17,participant1,Cool! What voices do you hear?,Human,0x14f93908
18,18,participant2,"i am a little conservative , but i do not have a lot of time to do .",Bot,0x14f93908
19,19,participant1,I understand. See you later!,Human,0x14f93908


In [ ]:
# Daro - Prueba con NLTK

import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

#sentence = """I love iphone! i just bought new iphone!"""
#tokens = nltk.word_tokenize(sentence)
#print(tokens)
#tagged = nltk.pos_tag(tokens)
#print(tagged)
#entities = nltk.chunk.ne_chunk(tagged)
#print(entities)


In [ ]:
# DARO - prueba con spacy

import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Apple Apple PROPN NNP nsubj Xxxxx True False
is be AUX VBZ aux xx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. U.K. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False


In [ ]:
from spacy.lang.en import English


def filter_stop_words(text):
  # Load English tokenizer, tagger, parser, NER and word vectors
  nlp = English()

  #  "nlp" Object is used to create documents with linguistic annotations.
  my_doc = nlp(text)

  # Create list of word tokens
  token_list = []
  for token in my_doc:
      token_list.append(token.text)

  from spacy.lang.en.stop_words import STOP_WORDS

  # Create list of word tokens after removing stopwords
  filtered_sentence =[] 

  for word in token_list:
      lexeme = nlp.vocab[word]
      if lexeme.is_stop == False:
          filtered_sentence.append(word) 
  return filtered_sentence


filtered_tokens_col = []
for value in df["text"]:
  filtered_tokens_col.append(filter_stop_words(value))
       
df["filtered_tokens_col"] = filtered_tokens_col





In [ ]:
# Import Drive API and authenticate.
from google.colab import drive

# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

# Write the DataFrame to CSV file.
df.to_csv('/gdrive/My Drive/df-filtered.csv')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
